# Word2Vec實作
- 字詞所代表的意義非常多元，在不同狀況下，會代表不同意思。要把多元意思用單一向量表示，則必須要進行word embedding的動作，也就是把高維向量降為低維向量的過程
- 之前介紹過，利用分散式表示法來表達字詞向量，例如PMI、SVD..統計法..等
- 2013年神經網路盛行後，Tomas Mikolov利用神經網路訓練方式，來獲得字詞的表達向量，獲得很棒的成果。一般認為是利用神經網路模擬人類的理解能力，獲得不錯的分布空間所得到的成果。
- 本範例以維基百科wiki部分資料作範例
- 資料來源：https://dumps.wikimedia.org/zhwiki/20231201/zhwiki-20231201-pages-articles-multistream1.xml-p1p187712.bz2
- 利用結巴分詞(jieba)進行斷詞，gensim套件進行word2vec計算
- 本範例約需1小時長時間執行


In [ ]:
!wget https://dumps.wikimedia.org/zhwiki/20240501/zhwiki-20240501-pages-articles-multistream1.xml-p1p187712.bz2 #運行下載命令

### opencc是繁簡轉換工具

In [ ]:
!pip install opencc-python-reimplemented #安裝 opencc-python-reimplemented 庫

### gensim是訓練word2vec的函式庫

In [ ]:
from gensim.corpora import WikiCorpus #導入 WikiCorpus 類

wiki_corpus = WikiCorpus('zhwiki-20240501-pages-articles-multistream1.xml-p1p187712.bz2', dictionary={}) #創建 WikiCorpus 實例

In [ ]:
wiki_corpus #創建了 wiki_corpus

In [ ]:
next(iter(wiki_corpus.get_texts()))[:10] #獲取語料庫中第一個文檔的前 10 個詞語

## 把wiki的資料檔案，轉換成連續文字的txt檔案

In [ ]:
text_num = 0

with open('wiki_text.txt', 'w', encoding='utf-8') as f:
    for text in wiki_corpus.get_texts():
        f.write(' '.join(text)+'\n')
        text_num += 1
        if text_num % 10000 == 0:
            print('{} articles processed.'.format(text_num))

    print('{} articles processed.'.format(text_num))# wiki_corpus.get_texts() 中獲取的每個文檔寫入到名為 wiki_text.txt 的文本文件

In [ ]:
import jieba
from opencc import OpenCC


# Initial
cc = OpenCC('s2t') #創建一個 OpenCC 的實例
train_data = open('wiki_text.txt', 'r', encoding='utf-8').read() #打開名為 wiki_text.txt 的文件，讀取其中的內容並將其賦值給變數 train_data
train_data = cc.convert(train_data) #使用 OpenCC 庫將 train_data 中的文本從簡體中文轉換為繁體中文
train_data = jieba.lcut(train_data) #使用 jieba 庫對文本進行分詞，將其拆分成一個詞語列表
train_data = [word for word in train_data if word != ''] #過濾掉列表中的空字符串
train_data = ' '.join(train_data) #將處理後的詞語列表重新組合為一個字串，詞語之間以空格分隔
open('seg.txt', 'w', encoding='utf-8').write(train_data) #打開名為 seg.txt 的文件，將處理後的文本寫入其中。這樣就完成了繁體中文分詞和繁簡轉換的處理

In [ ]:
from gensim.models import word2vec #導入 word2vec 模型


# Settings
seed = 666
sg = 0
window_size = 10
#vector_size = 100
min_count = 1
workers = 8
#epochs = 5
batch_words = 10000

train_data = word2vec.LineSentence('seg.txt') #用 word2vec.LineSentence 讀取包含分詞後文本的 seg.txt 文件
model = word2vec.Word2Vec( #創建 Word2Vec 模型
    train_data, #訓練數據，即分詞後的文本
    min_count=min_count, #設置最小詞頻
    #size=vector_size,
    workers=workers, #設置用於訓練的線程數
    #iter=epochs,
    window=window_size, #設置用於訓練的線程數
    sg=sg, #設置模型的訓練算法。如果 sg=0，則使用 CBOW 算法；如果 sg=1，則使用 Skip-gram 算法
    seed=seed, #設置隨機種子，用於初始化模型參數的隨機數生成
    batch_words=batch_words #設置每次訓練時使用的單詞數量
)

model.save('word2vec.model') #將訓練好的 Word2Vec 模型保存到名為 word2vec.model 的文件中

In [ ]:
from gensim.models import word2vec

string = '電腦' # 設置要查找的詞語
model = word2vec.Word2Vec.load('word2vec.model') #加載訓練好的 Word2Vec 模型
print(string) # 輸出要查找的詞語

for item in model.wv.most_similar(string): # 查找與該詞語最相似的詞語並輸出
    print(item)